In [135]:
!python -V

Python 3.6.9


In [136]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import feature_column

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from tensorflow.keras.utils import plot_model

import pandas as pd
from sklearn.model_selection import train_test_split


2.2.0-rc3


In [137]:
DATA_PATH = './TextDataset.csv'

#it is just two column csv, like:
# text;label
# A wiki is run using wiki software;0
# otherwise known as a wiki engine.;1

dataframe = pd.read_csv(DATA_PATH, delimiter = ';')
dataframe.head()

,text,label
0,A wiki is run using wiki software,0
1,otherwise known as a wiki engine.,1
2,A wiki engine is a type of content management ...,1
3,"but it differs from most other such systems,in...",0
4,in that the content is created without any def...,0


In [138]:
# Preprocessing before feature_clolumn includes
# - getting the vocabulary
# - tokenization, which means only splitting on tokens. Encoding sentences with vocablary will be done by feature_column!
# - padding
# - truncating

# Build vacabulary
vocab_size = 100
oov_tok = '<OOV>'

sentences = dataframe['text'].to_list()

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")

tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

# if word_index shorter then default value of vocab_size we'll save actual size
vocab_size=len(word_index)
print("vocab_size = word_index = ",len(word_index))

vocab_size = word_index =  59


In [0]:
# Split sentensec on tokens. here token = word
# text_to_word_sequence() has good default filter for charachters include basic punctuation, tabs, and newlines
dataframe['text'] = dataframe['text'].apply(tf.keras.preprocessing.text.text_to_word_sequence)

In [140]:
dataframe.head()

,text,label
0,"[a, wiki, is, run, using, wiki, software]",0
1,"[otherwise, known, as, a, wiki, engine]",1
2,"[a, wiki, engine, is, a, type, of, content, ma...",1
3,"[but, it, differs, from, most, other, such, sy...",0
4,"[in, that, the, content, is, created, without,...",0


In [141]:
max_length = 6

# paddind and trancating setnences
# do that directly with strings without using tokenizer.texts_to_sequences()
# the feature_colunm will convert strings into numbers
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: (x + N * [''])[:N])
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: x[:N])
dataframe.head()

,text,label
0,"[a, wiki, is, run, using, wiki]",0
1,"[otherwise, known, as, a, wiki, engine]",1
2,"[a, wiki, engine, is, a, type]",1
3,"[but, it, differs, from, most, other]",0
4,"[in, that, the, content, is, created]",0


In [0]:
# Define method to create tf.data dataset from Pandas Dataframe
# This worked with tf 2.0 but does not work with tf 2.2
def df_to_dataset_tf_2_0(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    #labels = dataframe.pop(label_column)
    labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

def df_to_dataset(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop(label_column)
    #labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dataframe .to_dict(orient='list'), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds



   

In [143]:
# Split dataframe into train and validation sets
train_df, val_df = train_test_split(dataframe, test_size=0.2)

print(len(train_df), 'train examples')
print(len(val_df), 'validation examples')

7 train examples
2 validation examples


In [0]:
batch_size = 32
ds = df_to_dataset(dataframe, 'label',shuffle=False,batch_size=batch_size)

train_ds = df_to_dataset(train_df, 'label', batch_size=batch_size)
val_ds = df_to_dataset(val_df, 'label', shuffle=False, batch_size=batch_size)

In [145]:
# and small batch for demo
example_batch = next(iter(ds))[0]
example_batch

{'text': <tf.Tensor: shape=(9, 6), dtype=string, numpy=
 array([[b'a', b'wiki', b'is', b'run', b'using', b'wiki'],
        [b'otherwise', b'known', b'as', b'a', b'wiki', b'engine'],
        [b'a', b'wiki', b'engine', b'is', b'a', b'type'],
        [b'but', b'it', b'differs', b'from', b'most', b'other'],
        [b'in', b'that', b'the', b'content', b'is', b'created'],
        [b'and', b'wikis', b'have', b'little', b'inherent', b'structure'],
        [b'allowing', b'structure', b'to', b'emerge', b'according', b'to'],
        [b'there', b'are', b'dozens', b'of', b'different', b'wiki'],
        [b'both', b'standalone', b'and', b'part', b'of', b'other']],
       dtype=object)>}

In [0]:
# Helper method to print exxample outputs of for defined feature_column

def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [147]:
# Define categorical colunm for our text feature, which is preprocessed into lists of tokens
# Note that key name should be the same as original column name in dataframe
text_column = feature_column.categorical_column_with_vocabulary_list(key='text', 
                                                                     vocabulary_list=list(word_index))

#indicator_column produce one-hot-encoding. These lines just to compare with embedding
#print(demo(feature_column.indicator_column(payment_description_3)))
#print(payment_description_2,'\n')

# arguemnt dimention here is exactly the dimension of the space in which tokens will be presented during model's learning
# see the tutorial at https://www.tensorflow.org/beta/tutorials/text/word_embeddings
text_embedding = feature_column.embedding_column(text_column, dimension=8)
print(demo(text_embedding))

[[-0.1245741  -0.06539811 -0.04908126  0.22344409 -0.08359805 -0.06485327
   0.03772326 -0.10539886]
 [-0.14375462 -0.02242014 -0.08543857 -0.06870032 -0.12047953  0.09668547
  -0.02564205  0.19334948]
 [-0.07704347 -0.23529786  0.00991495  0.12841998  0.00426027 -0.0811061
   0.1292375   0.13235547]
 [-0.16376287 -0.2243027  -0.14932789  0.15451281 -0.07146135  0.13924138
  -0.15498729 -0.01912173]
 [ 0.01838289  0.18435316 -0.04038678 -0.03248038  0.2950769   0.01126851
  -0.20305036  0.05689808]
 [-0.08320137 -0.13970047 -0.07709267  0.19206601  0.15707016 -0.04501485
   0.00832598 -0.08314264]
 [-0.09318731  0.03975042  0.07772637 -0.13613771 -0.0662718   0.32315478
   0.01094771 -0.14336602]
 [ 0.12388621  0.20748846 -0.06411789  0.11938087 -0.04696181 -0.02893485
   0.28302118  0.13568495]
 [-0.04980196  0.11031214 -0.12780045  0.23032922 -0.15213704 -0.03006172
  -0.05110123  0.10465026]]
None


In [0]:
# The define the layers and model it self
# This example uses Keras Functional API instead of Sequential just for more generallity

# Define DenseFeatures layer to pass feature_columns into Keras model
feature_layer = tf.keras.layers.DenseFeatures(text_embedding)

In [149]:
# Define inputs for each feature column. See
# см. https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673
feature_layer_inputs = {}

# Here we have just one column
feature_layer_inputs['text'] = tf.keras.Input(shape=(max_length,), name='text', dtype=tf.string)
print(feature_layer_inputs)

{'text': <tf.Tensor 'text_3:0' shape=(None, 6) dtype=string>}


In [150]:
# Define outputs of DenseFeatures layer 
# And accually use them as first layer of the model
feature_layer_outputs = feature_layer(feature_layer_inputs)
print(feature_layer_outputs)

# Add consequences layers. See https://keras.io/getting-started/functional-api-guide/
x = tf.keras.layers.Dense(256, activation='relu')(feature_layer_outputs)
x = tf.keras.layers.Dropout(0.2)(x)

# This example supposes binary classification, as labels are 0 or 1
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=x)

model.summary()

# This example supposes binary classification, as labels are 0 or 1
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              #run_eagerly=True
             )

Tensor("dense_features_9/Identity:0", shape=(None, 8), dtype=float32)
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            [(None, 6)]               0         
_________________________________________________________________
dense_features_9 (DenseFeatu (None, 8)                 472       
_________________________________________________________________
dense_10 (Dense)             (None, 256)               2304      
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 3,033
Trainable params: 3,033
Non-trainable params: 0
_________________________________________________________________


In [151]:
# Note that fit() method looking up features in train_ds and valdation_ds by name in 
# tf.keras.Input(shape=(max_length,), name='text'

# This model of cause will learn nothing because of fake data.

num_epochs = 5
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=num_epochs,
                    verbose=1
                    )

Epoch 1/5
1/1 [==============================] - 0s 136ms/step - loss: 0.6911 - accuracy: 0.5714 - val_loss: 0.6983 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 19ms/step - loss: 0.6932 - accuracy: 0.4286 - val_loss: 0.7022 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 11ms/step - loss: 0.6884 - accuracy: 0.7143 - val_loss: 0.7062 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 12ms/step - loss: 0.6864 - accuracy: 0.7143 - val_loss: 0.7101 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 10ms/step - loss: 0.6816 - accuracy: 0.7143 - val_loss: 0.7141 - val_accuracy: 0.0000e+00
